In [ ]:

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/solar_forecasting')

if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !pip3 install timm==0.4.5  # 0.3.2 does not work in Colab



import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import os
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

Mounted at /content/drive
Running in Colab.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation:

In [ ]:
#pretraining  images

image_path = '/content/drive/MyDrive/solar_forecasting/images/' # where to load images from
video_path = '/content/drive/MyDrive/solar_forecasting/videos/' # where to save the videos
data_path = '/content/drive/MyDrive/solar_forecasting/data/2017_pv_raw.csv' # file to load labels from (with filename)
label_path = '/content/drive/MyDrive/solar_forecasting/data/' # where to save the organized labels (without filename)


SIZE = 224 # image size
FRAMES_PER_VIDEO = 16 # number of frames in video
year = "2017"

TIME_BETWEEN_FRAMES = 1 # minutes
FPS = 1 #1 / (60 * TIME_BETWEEN_FRAMES)

time_intervals = [5, 10, 15, 20, 25, 30] # times to predict pv output for (during finetuning)



"""
def loop_through_folders(directory):
    for path, folders, files in os.walk(directory):
        for folder_name in folders:
            print(f"Content of '{folder_name}':")
            print(os.listdir(f"{path}/{folder_name}"))

loop_through_folders(image_path)
"""

'\ndef loop_through_folders(directory):\n    for path, folders, files in os.walk(directory):\n        for folder_name in folders:\n            print(f"Content of \'{folder_name}\':")\n            print(os.listdir(f"{path}/{folder_name}"))\n\nloop_through_folders(image_path)\n'

In [ ]:
pv_outputs = pd.read_csv(data_path)
pv_outputs['Date'] = pv_outputs['Date'].astype(str)

columns = ['video_path']
for i in range(len(time_intervals)):
  columns.append('pv_' + str(time_intervals[i]))

print(pv_outputs)

In [ ]:
#TODO: allow time between frames of more than 1 minute


def build_video_dataset(root, finetune=False, columns=None):
  # assumes images in a given day are in chronological order
  # if finetune is true, find and save pv output labels at the same time
  label_list = []
  for month in os.listdir(root):
    path1 = os.path.join(root, month)
    if not os.path.isdir(path1):
      continue
    for day in os.listdir(path1):
      path2 = os.path.join(path1, day)
      images = []
      # loop through images, concat every 16
      for image in os.listdir(path2):
        images.append(image)
        if len(images) == FRAMES_PER_VIDEO:
            video_time = images[-1][:-6] # excluding seconds
            video_name = video_path + video_time + '00.mp4'
            if finetune:
              video_time = video_time[0:4] + "-" + video_time[4:6] + "-" + video_time[6:8] + "T" + video_time[8:10] + ":" + video_time[10:12] + ":00"
              index = pv_outputs.index[pv_outputs['Date']==video_time]
              if (len(index) == 0):
                continue
              pvs = [video_name]
              for i in range(len(time_intervals)):
                pv_index = index[0] + time_intervals[i]
                if pv_index >= len(pv_outputs):
                  pvs.append(np.nan)
                else:
                  pvs.append(pv_outputs.iloc[pv_index]['Huang_E4102_kW'])
              label_list.append(pvs)
              print(video_name + ": Label added successfully!")
            video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), FPS, (SIZE, SIZE))
            for im in images:
              image = cv2.imread(os.path.join(path2, im))
              resized_image = cv2.resize(image, (SIZE, SIZE), interpolation=cv2.INTER_AREA)
              video.write(resized_image)
            video.release()
            cv2.destroyAllWindows()
            print(video_name + ": Video generated successfully!")
            images = []
  if finetune:
    labels = pd.DataFrame(label_list, columns=columns)
    labels.to_csv(label_path + 'labels.csv')
    print("Labels saved successfully!")


build_video_dataset(image_path, finetune=True, columns=columns)

In [ ]:
#save labels into separate files for each time interval

labels = pd.read_csv(label_path + 'labels.csv')
for time in time_intervals:
  labels.to_csv(label_path + 'labels_' + str(time) + '.csv', columns=['video_path', 'pv_'+str(time)], index=False)


In [ ]:
# split into train/val/test
for time in time_intervals:
  labels = pd.read_csv(label_path + 'labels_' + str(time) + '.csv')
  shuffled = labels.sample(frac=1)
  result = np.array_split(shuffled, 5)
  train = pd.concat([result[0], result[1], result[2]])
  val = result[3]
  test = result[4]
  if not os.path.exists(label_path + str(time)):
    os.makedirs(label_path + str(time))
  train.to_csv(label_path + str(time) + '/train.csv')
  val.to_csv(label_path + str(time) + '/val.csv')
  test.to_csv(label_path + str(time) + '/test.csv')
  print("Split successfully!")

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Split successfully!
Split successfully!
Split successfully!
Split successfully!
Split successfully!
Split successfully!


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
